# School coverage analysis for Bogotá

The purpose of this project is to do an exploration of the school coverage in Bogotá using official information from the education ministry. The used dataset can be found in the open data page of the colombian government in the following <a href="https://www.datos.gov.co/Educaci-n/LISTADO-COLEGIOS-BOGOTA/qijw-htwa">link</a>. The dataset is a little outdated, as it has information from 2016, but the analysis should mostly hold as education coverage does not change rapidly.

## Imports

In [15]:
import pandas as pd
import numpy as np
import geopandas as gpd
import pandas as pd
from googlemaps import Client as GoogleMaps
import googlemaps
import json
import plotly.express as px
from sklearn.cluster import DBSCAN

<IPython.core.display.Javascript object>

In [2]:
%load_ext nb_black

<IPython.core.display.Javascript object>

## EDA and preprocessing

In [78]:
school_data = pd.read_csv('../data/LISTADO_COLEGIOS_BOGOTA.csv')

<IPython.core.display.Javascript object>

In [79]:
for c in school_data.columns:
    if len(school_data[c].unique()) < 2:
        print(f'Column {c} dropped for only having one value')
        school_data.drop(c, axis=1, inplace=True)

Column año dropped for only having one value
Column secretaria dropped for only having one value
Column sector dropped for only having one value
Column genero dropped for only having one value
Column caracter dropped for only having one value
Column estado dropped for only having one value
Column internado dropped for only having one value


<IPython.core.display.Javascript object>

In [80]:
school_data.head()

,nombreestablecimiento,zona,direccion,telefono,nombre_Rector,tipo_Establecimiento,etnias,niveles,jornadas,especialidad,...,discapacidades,idiomas,numero_de_Sedes,prestador_de_Servicio,propiedad_Planta_Fisica,resguardo,matricula_Contratada,calendario,estrato_Socio_Economico,correo_Electronico
0,INST DE BTO TEC COMERCIAL PITAGORAS ...,URBANA,KR 5 11 67,5665677 / 3429092,YOLANDA ARIAS CRUZ,INSTITUCION EDUCATIVA,NaN,"MEDIA,BÁSICA SECUNDARIA",COMPLETA,"COMERCIAL,ACADÉMICA",...,NaN,INGLÉS,1,PERSONA NATURAL,PERSONA NATURAL,NO APLICA,NO,A,NaN,pitagoras@pitagoras.edu.co
1,GIMNASIO INFANTIL FEDERICO FROEBEL,URBANA,CL 32 SUR 52 A 54,7100349,SANDRA MILENA MENDEZ,CENTRO EDUCATIVO,NaN,PREESCOLAR,COMPLETA,NaN,...,NaN,NaN,1,PERSONA NATURAL,PERSONA NATURAL,NO APLICA,NO,A,NaN,gimfedebel@hotmail.com
2,GIMNASIO INFANTIL LOS NIÑOS DEL MAÑANA,URBANA,CL 134 D 47 27,6010350,CLAUDIA BOCACHICA SAENZ,INSTITUCION EDUCATIVA,NaN,"PREESCOLAR,BÁSICA SECUNDARIA,BÁSICA PRIMARIA",COMPLETA,NaN,...,NaN,NaN,1,PERSONA NATURAL,PERSONA NATURAL,NO APLICA,NO,A,NaN,claudialindenibocachica@hotmail.com
3,COLEGIO FRANCISCO PRIMERO S.S. (IED),URBANA,KR 28 63 64,2126019 -- 2481271,JOSE CUSTODIO GALARZA MORENO,INSTITUCION EDUCATIVA,NaN,"PREESCOLAR,MEDIA,BÁSICA SECUNDARIA,BÁSICA PRIM...","MAÑANA,ÚNICA,TARDE",ACADÉMICA,...,NaN,NaN,4,OFICIAL,OFICIAL,NO APLICA,NO,A,NaN,cedalemania12@redp.edu.co
4,COL SAN JUAN VIANETH ...,URBANA,KR 2 C 3 18 SUR,2804738 -- 4714934,GLORIA ESPERANZA CHUQUIN LAISECA,CENTRO EDUCATIVO,NaN,"PREESCOLAR,BÁSICA PRIMARIA","MAÑANA,COMPLETA",NaN,...,NaN,INGLÉS,1,PERSONA NATURAL,PERSONA NATURAL,NO APLICA,NO,A,NaN,colegiosanjuanviateh@hotmail.com -- colegiosan...


<IPython.core.display.Javascript object>

In [81]:
school_data.iloc[0]

nombreestablecimiento        INST DE BTO TEC COMERCIAL PITAGORAS           ...
zona                                                                    URBANA
direccion                                                           KR 5 11 67
telefono                                                   5665677   / 3429092
nombre_Rector                                               YOLANDA ARIAS CRUZ
tipo_Establecimiento                                     INSTITUCION EDUCATIVA
etnias                                                                     NaN
niveles                                                MEDIA,BÁSICA SECUNDARIA
jornadas                                                              COMPLETA
especialidad                                               COMERCIAL,ACADÉMICA
grados                                                           6,7,8,9,10,11
modelos_Educativos                                       EDUCACIÓN TRADICIONAL
capacidades_Excepcionales                           

<IPython.core.display.Javascript object>

We are not interested in information about the name, telephone number, ethnic status, schedulle type, especiality, and other attributes of the schools, so this columns are als dropped.

In [82]:
school_data.drop(['telefono', 'especialidad', 'grados', 'modelos_Educativos','etnias',
       'capacidades_Excepcionales', 'discapacidades', 'numero_de_Sedes','propiedad_Planta_Fisica',
       'resguardo', 'matricula_Contratada', 'calendario','nombre_Rector','tipo_Establecimiento',
       'estrato_Socio_Economico', 'correo_Electronico','jornadas'], axis = 1, inplace=True)

<IPython.core.display.Javascript object>

In [83]:
school_data.head()

,nombreestablecimiento,zona,direccion,niveles,idiomas,prestador_de_Servicio
0,INST DE BTO TEC COMERCIAL PITAGORAS ...,URBANA,KR 5 11 67,"MEDIA,BÁSICA SECUNDARIA",INGLÉS,PERSONA NATURAL
1,GIMNASIO INFANTIL FEDERICO FROEBEL,URBANA,CL 32 SUR 52 A 54,PREESCOLAR,NaN,PERSONA NATURAL
2,GIMNASIO INFANTIL LOS NIÑOS DEL MAÑANA,URBANA,CL 134 D 47 27,"PREESCOLAR,BÁSICA SECUNDARIA,BÁSICA PRIMARIA",NaN,PERSONA NATURAL
3,COLEGIO FRANCISCO PRIMERO S.S. (IED),URBANA,KR 28 63 64,"PREESCOLAR,MEDIA,BÁSICA SECUNDARIA,BÁSICA PRIM...",NaN,OFICIAL
4,COL SAN JUAN VIANETH ...,URBANA,KR 2 C 3 18 SUR,"PREESCOLAR,BÁSICA PRIMARIA",INGLÉS,PERSONA NATURAL


<IPython.core.display.Javascript object>

Now that we have filtered all the attributes of interest let's explore each column and translate its meaning to english.

In [84]:
school_data['zona'].unique()

array(['URBANA', 'RURAL', 'RURAL,URBANA'], dtype=object)

<IPython.core.display.Javascript object>

The attribute zona classifies the location of the school between urban and rural. For this particular analysis we are only interested in the urban area of Bogotá, so let's retain only those institutions.

In [85]:
school_data = school_data[school_data['zona'] == 'URBANA'].drop('zona', axis=1)

<IPython.core.display.Javascript object>

The niveles feature presents the educatinal levels available at the different schoools. The next code cell presents the translation of the different levels and process the column into one binary attribute for each level.

In [86]:
school_data['niveles'] = school_data['niveles'].apply(lambda x: [] if pd.isna(x) else x.split(','))
set(np.concatenate(school_data['niveles'].values))

{'BÁSICA PRIMARIA',
 'BÁSICA SECUNDARIA',
 'MEDIA',
 'PREESCOLAR',
 'PRIMERA INFANCIA'}

<IPython.core.display.Javascript object>

In [87]:
dic_translate_level = {
    'BÁSICA PRIMARIA': 'Primary',
    'BÁSICA SECUNDARIA': 'Secondary',
    'MEDIA': 'Middle',
    'PREESCOLAR': 'Preschool',
    'PRIMERA INFANCIA': 'Childcare'
}
for k,v in dic_translate_level.items():
    school_data[v] = school_data['niveles'].apply(lambda x: k in x)
school_data.drop('niveles',axis=1,inplace=True)

<IPython.core.display.Javascript object>

The idiomas attribute has information about the languages different from spanish taught in the educational institutes. From this attribute let's create another one that encode if the school is bilingual or not.

In [88]:
school_data['bilingual'] = ~school_data['idiomas'].isna()
school_data.drop('idiomas', axis=1, inplace=True)

<IPython.core.display.Javascript object>

Finally from the prestador_de_Servicio attribute we are going to extract if the school is public or private and save it on a feature named public.

In [89]:
school_data['public'] = school_data['prestador_de_Servicio'] == 'OFICIAL'
school_data.drop('prestador_de_Servicio', axis=1, inplace=True)

<IPython.core.display.Javascript object>

Now that we processed attributes of interest let's explore the resulting data frame.

In [90]:
school_data.drop('direccion',axis=1).mean()

/var/folders/sy/55_grszs60j_67x7hfzqk8800000gn/T/ipykernel_16068/1850020645.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  school_data.drop('direccion',axis=1).mean()


Primary      0.760759
Secondary    0.582278
Middle       0.546835
Preschool    0.886498
Childcare    0.002110
bilingual    0.395781
public       0.160759
dtype: float64

<IPython.core.display.Javascript object>

The most common schools are preschools and primary schools, while the number of institutions that serve as childcare for babies is relatively small.

In [91]:
np.corrcoef(school_data['bilingual'],school_data['public'])[0,1]

-0.354221572326856

<IPython.core.display.Javascript object>

There is a negative correlation between the instruction of a second language and the attribute that tells if the school is public. In other words, public schools are less likely to be bilingual.

## Geocoding

In order to do the coverage analysis we require to convert the addresses to geocodified location with latitude and longitude. To do this we are going to use the google maps api. In order to run the notebook you need to generate a api key and save it in keys.json under the key maps_key.

In [92]:
vc_address = school_data['direccion'].value_counts()

<IPython.core.display.Javascript object>

As the following table shows, there are 49 schools that share their address with another one. After some exploration it was found that some of these addresses were wrong. Because the number of schools for which the incorrect address is small, a manual correction was done for these 49 shools.

In [99]:
school_data[school_data['direccion'].isin(vc_address[vc_address>1].index)].sort_values('direccion')[
    ['nombreestablecimiento','direccion']
].head(10)

,nombreestablecimiento,direccion
517,COLEGIO LA TOSCANA - LISBOA (IED),CL 132 D 132 05
687,COLEGIO GONZALO ARANGO (IED),CL 132 D 132 05
26,COLEGIO INTEGRADO EDUARDO CABALLERO CALDERON,CL 132 D BIS 132 08
1004,COLEGIO SUPERIOR DE PALERMO,CL 132 D BIS 132 08
1507,INST CULT RAFAEL MAYA ...,CL 134 A 147 C 07
2244,CENTRO JOHANN KEPLER,CL 134 A 147 C 07
1966,COL CIAL VILLA MARIA ...,CL 139 A 111 04
220,CENT DE EDUC DE ADULTOS CULTURAL,CL 139 A 111 04
1464,FUNDACIÓN DE EDUCACION EDUARDO ESCOVAR GONZALE...,CL 146 A 94 D 19
1045,CENT PANAMERICANO DE CAPACITACION SEDE SUBA ...,CL 146 A 94 D 19


<IPython.core.display.Javascript object>

In [100]:
school_data[school_data['direccion'].isin(vc_address[vc_address>1].index)].sort_values('direccion')[
    ['nombreestablecimiento','direccion']
].to_csv('../data/repeated_addresses.csv')

<IPython.core.display.Javascript object>

In [104]:
repeated_address_depuration = pd.read_csv('../data/repeated_addresses_corrected.csv',index_col=0)

<IPython.core.display.Javascript object>

In [111]:
school_data = pd.merge(
    school_data,
    repeated_address_depuration.drop(['direccion','nombreestablecimiento'],axis=1),
    left_index=True,
    right_index=True,
    how='left'
)

<IPython.core.display.Javascript object>

In [113]:
school_data = school_data[school_data['delete']!=1].drop('delete',axis=1).copy()

<IPython.core.display.Javascript object>

In [116]:
school_data['direccion'] = school_data.apply(
    lambda row: row['direccion'] if pd.isna(row['corrected_address']) else row['corrected_address'],
    axis=1
)

<IPython.core.display.Javascript object>

In [41]:
with open('../keys.json','r') as f:
    maps_key = json.load(f)['maps_key']

<IPython.core.display.Javascript object>

In [42]:
gmaps_client = googlemaps.Client(key=maps_key)

<IPython.core.display.Javascript object>

In [65]:
address_dic = {}
for address in school_data['direccion']:
    try:
        address_dic[address] = gmaps_client.geocode(address + ', Bogota, Colombia')[0]['geometry']['location']
    except:
        print(f'Request failed {address}')

Request failed CL 54 A 27 39 SUR


<IPython.core.display.Javascript object>

Let's retry the geocoding for the addresses that failed

In [118]:
for address in school_data['direccion']:
    if address not in address_dic.keys():
        try:
            address_dic[address] = gmaps_client.geocode(address + ', Bogota, Colombia')[0]['geometry']['location']
        except:
            print(f'Request failed {address}')

<IPython.core.display.Javascript object>

Now we can convert the addresses into latitude and longitude values

In [124]:
school_data['lat'] = school_data['direccion'].apply(lambda x: address_dic[x]['lat'])
school_data['lon'] = school_data['direccion'].apply(lambda x: address_dic[x]['lng'])

<IPython.core.display.Javascript object>

In [126]:
school_data.drop(['direccion','corrected_address'],axis=1,inplace=True)

<IPython.core.display.Javascript object>

Now that we have a processed dataset with categories and coordinate locations for each school, let's plot a map of the different institutions.

In [7]:
# school_data.to_csv('../data/school_data_processed.csv')
school_data = pd.read_csv("../data/school_data_processed.csv", index_col=0)

<IPython.core.display.Javascript object>

In [8]:
with open("../keys.json", "r") as f:
    mapbox_key = json.load(f)["mapbox"]

<IPython.core.display.Javascript object>

In [9]:
school_data_gpd = gpd.GeoDataFrame(
    school_data.drop(["lat", "lon"], axis=1),
    geometry=gpd.points_from_xy(school_data["lon"], school_data["lat"]),
)

<IPython.core.display.Javascript object>

In [10]:
school_data_gpd

,nombreestablecimiento,Primary,Secondary,Middle,Preschool,Childcare,bilingual,public,geometry
0,INST DE BTO TEC COMERCIAL PITAGORAS ...,False,True,True,False,False,True,False,POINT (-74.07331 4.59777)
1,GIMNASIO INFANTIL FEDERICO FROEBEL,False,False,False,True,False,False,False,POINT (-74.12871 4.60508)
2,GIMNASIO INFANTIL LOS NIÑOS DEL MAÑANA,True,True,False,True,False,False,False,POINT (-74.05455 4.72102)
3,COLEGIO FRANCISCO PRIMERO S.S. (IED),True,True,True,True,False,False,True,POINT (-74.07563 4.65213)
4,COL SAN JUAN VIANETH ...,True,False,False,True,False,True,False,POINT (-74.06367 4.61694)
...,...,...,...,...,...,...,...,...,...
2399,COLEGIO SAN JOSE DE CALASANZ,True,True,True,True,False,False,False,POINT (-74.08836 4.72321)
2400,COL NUEVO REINO DE TURINGIA,True,False,False,True,False,True,False,POINT (-74.09132 4.75305)
2401,LIC INF GOOFY ...,True,False,False,True,False,True,False,POINT (-74.18504 4.60156)
2402,LIC APRENDO CON MIS AMIGOS ...,True,False,False,True,False,True,False,POINT (-74.13807 4.67083)


<IPython.core.display.Javascript object>

In [14]:
px.set_mapbox_access_token(mapbox_key)
fig = px.scatter_geo(
    school_data_gpd,
    lat=school_data_gpd.geometry.y,
    lon=school_data_gpd.geometry.x,
    hover_name="nombreestablecimiento",
    color="public",
)
fig.update_geos(fitbounds="locations")
fig.update_layout(
    mapbox_style="open-street-maps",
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
    mapbox_accesstoken=mapbox_key,
)
fig.show()

<IPython.core.display.Javascript object>

From the map above it can be observed a greater concentration of public schools in the south of the city which correlates with the poorest zones of Bogotá.

## Coverage

In order to explore the coverage we are going to use the DBscan algorithm. This density based clustering algorithm is based on two parameters $\epsilon$ and n_points which jointly define the necesary density for a cluster. DBScan identifies the points that have at least n_points closer than $\epsilon$ and clasify them as core points. Every core point with its $\epsilon$ neighbors form a cluster which is then extended by the neighbors of other core points inside the cluster. For more information about this algorithm can be found in the sklearn <a href="https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html">documentation</a>.

In order to apply the algorithm we have to project the geospatial data and define $\epsilon$ and n_features. Considering that the school locations are all in Bogotá, and that Colombia is a ecuatorial country (if not longitude meassures would be greatly distorted), we can safely do the naive projection of lat and lon into the plane. Let's now do a first analysis requiring at least 4 schools in a radius of 500 meters in order to be considered a core point

In [58]:
db_pred = DBSCAN(eps=1 / (111.1 * 2), min_samples=4).fit_predict(
    school_data[["lat", "lon"]]
)

<IPython.core.display.Javascript object>

In [59]:
school_data_gpd["cluster_all"] = db_pred

<IPython.core.display.Javascript object>

In [79]:
def plot_dbscan(col, school_data_gpd=school_data_gpd):
    school_data_gpd["member"] = (school_data_gpd[col] >= 0).apply(
        lambda x: 1 if x else 0.3
    )
    px.set_mapbox_access_token(mapbox_key)
    fig = px.scatter_geo(
        school_data_gpd,
        lat=school_data_gpd.geometry.y,
        lon=school_data_gpd.geometry.x,
        hover_name="nombreestablecimiento",
        color=col,
        size="member",
        size_max=5,
    )
    fig.update_geos(fitbounds="locations")
    fig.update_layout(
        mapbox_style="open-street-maps",
        margin={"r": 0, "t": 0, "l": 0, "b": 0},
        mapbox_accesstoken=mapbox_key,
    )
    fig.show()

<IPython.core.display.Javascript object>

In [80]:
plot_dbscan('cluster_all')

<IPython.core.display.Javascript object>

The first map shows that there is a low concentration of schools as expected in natural resources like the suba wetlands and hill, and in the undeveloped parts of Usme. The map also shows that there is also low school coverage in Usaquen and in the industrial zone around Cra 13. Let strengthen the coverage requirements by asking for at least 8 schools in a 500 m radius.

In [71]:
school_data_gpd["cluster_all_2"] = DBSCAN(
    eps=1 / (111.1 * 2), min_samples=8
).fit_predict(school_data[["lat", "lon"]])

<IPython.core.display.Javascript object>

In [81]:
plot_dbscan("cluster_all_2")

<IPython.core.display.Javascript object>

By increasing the density requirements new problematic areas arise like Kennedy in the south west of the city and the east of Suba. Now let's complement the analysis by considering some subsets of the schools.

### Public schools

In [76]:
school_data_gpd_analysis = school_data_gpd[school_data_gpd["public"]].copy()
school_data_analysis = school_data[school_data["public"]].copy()

<IPython.core.display.Javascript object>

In [83]:
school_data_gpd_analysis["cluster"] = DBSCAN(eps=1 / (111.1), min_samples=4).fit_predict(
    school_data_analysis[["lat", "lon"]]
)

<IPython.core.display.Javascript object>

In [84]:
plot_dbscan("cluster", school_data_gpd_analysis)

<IPython.core.display.Javascript object>

When considering only public schools we can see that there is a lack of coverage in the north and west of the city. The low density in the north of the city can be explained by the greater income of citizens in such zone and the preference for private schools, but the lack of public institutions around Cra 26  and Cra 13 is problematic and should be explored further.

### Bilingual schools

In [93]:
school_data_gpd_analysis = school_data_gpd[school_data_gpd["bilingual"]].copy()
school_data_analysis = school_data[school_data["bilingual"]].copy()

<IPython.core.display.Javascript object>

In [112]:
school_data_gpd_analysis["cluster"] = DBSCAN(
    eps=1 / (111.1), min_samples=8
).fit_predict(school_data_analysis[["lat", "lon"]])

<IPython.core.display.Javascript object>

In [113]:
plot_dbscan("cluster", school_data_gpd_analysis)

<IPython.core.display.Javascript object>

The density of bilingual schools is similar to the density of all schools as shown in the map above with an important difference in Usme where there is low concentration of these type of schools.

### Preschools

In [119]:
school_data_gpd_analysis = school_data_gpd[school_data_gpd["Preschool"]].copy()
school_data_analysis = school_data[school_data["Preschool"]].copy()

<IPython.core.display.Javascript object>

In [132]:
school_data_gpd_analysis["cluster"] = DBSCAN(
    eps=1 / (111.1 * 2), min_samples=8
).fit_predict(school_data_analysis[["lat", "lon"]])

<IPython.core.display.Javascript object>

In [133]:
plot_dbscan("cluster", school_data_gpd_analysis)

<IPython.core.display.Javascript object>

The dbscan result for preschools is similar to the result for all schools, but shos an important difference in Chapinero and Usaquen where the density is very low in relation to the rest of the city.

### Secondary schools

In [135]:
school_data_gpd_analysis = school_data_gpd[school_data_gpd["Secondary"]].copy()
school_data_analysis = school_data[school_data["Secondary"]].copy()

<IPython.core.display.Javascript object>

In [142]:
school_data_gpd_analysis["cluster"] = DBSCAN(
    eps=1 / (111.1 * 2), min_samples=4
).fit_predict(school_data_analysis[["lat", "lon"]])

<IPython.core.display.Javascript object>

In [143]:
plot_dbscan("cluster", school_data_gpd_analysis)

<IPython.core.display.Javascript object>

There are much less secondary schools in Bogotá than preschools. The deficit of this type of schools is bigger in the industrial zone and the north of the city.

## Conclusions

Using the dbscan algorithm we found that the industrial zone of Bogotá and Usaquen have low educational coverage in relation with the rest of the city. The analysis also showed that public schools are concentrated in the south of the city and there is a lack of bilingual schools in that area.

A little detail that would help greatly for map interpretation is to include a basemap with information about the city. Also, in order to improve the analysis we should correlate our findings with the population density in Bogotá and consider the size of the different schools. 